In [43]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [91]:
df = spark.read.parquet("data/final/merge_jugadores.parquet")
df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Born: float (nullable = true)
 |-- MP: float (nullable = true)
 |-- Starts: float (nullable = true)
 |-- Min: float (nullable = true)
 |-- Gls: float (nullable = true)
 |-- Ast: float (nullable = true)
 |-- YellowC: float (nullable = true)
 |-- RedC: float (nullable = true)
 |-- PrgC: float (nullable = true)
 |-- PrgP: float (nullable = true)
 |-- PrgR: float (nullable = true)
 |-- Fls: float (nullable = true)
 |-- Off: float (nullable = true)
 |-- Crosses: float (nullable = true)
 |-- Recov: float (nullable = true)
 |-- Aerialwon%: float (nullable = true)
 |-- Tack_Def_3rd: float (nullable = true)
 |-- Tack_Mid_3rd: float (nullable = true)
 |-- Tack_Att_3rd: float (nullable = true)
 |-- Tkl%: float (nullable = true)
 |-- Tkl: float (nullable = true)
 |-- TklW: float (nullable = true)
 |-- Int: float (nullable = true)
 |--

In [55]:
df.count()

4969

In [40]:
from pyspark.sql.functions import col



df_union = df_union.toPandas()

df_union.to_csv("data/unidos2.csv", index=False)

In [86]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Definir una ventana por Posición, ordenada por adjusted_score descendente
window_spec = Window.partitionBy("Pos").orderBy(col("adjusted_score").desc())
df_2024_2025 = df.filter(col("Season") == "2024-2025")
# Aplicar ranking
df_ranked = df.withColumn("rank", rank().over(window_spec))

# Filtrar los mejores (rank = 1) por posición
df_top_by_position = df_ranked.filter(col("rank") == 1)

# Mostrar resultados
df_top_by_position.select(
    "Pos", "Player", "Squad", "Season", "adjusted_score", "performance_score", "penalty_score", "Value"
).orderBy("Pos").show(truncate=False)


{"ts": "2025-08-05 19:41:20.760", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `adjusted_score` cannot be resolved. Did you mean one of the following? [`penalty_score`, `Crosses_stp%`, `Crosses`, `Crosses_Opp`, `Pass_Short`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor233.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o59951.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `adjusted_score` cannot be resolved. Did you mean one of the following? [`penalty_score`, `Crosses_stp%`, `Crosses`, `Crosses_Opp`, `Pass_Short`]. SQLSTATE: 42703;\n'Project [Player#71814, Squad#71815, Season#71816, Competition#71817, Born

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `adjusted_score` cannot be resolved. Did you mean one of the following? [`penalty_score`, `Crosses_stp%`, `Crosses`, `Crosses_Opp`, `Pass_Short`]. SQLSTATE: 42703;
'Project [Player#71814, Squad#71815, Season#71816, Competition#71817, Born#71818, MP#71819, Starts#71820, Min#71821, Gls#71822, Ast#71823, YellowC#71824, RedC#71825, PrgC#71826, PrgP#71827, PrgR#71828, Fls#71829, Off#71830, Crosses#71831, Recov#72027, Aerialwon%#71833, Tack_Def_3rd#71834, Tack_Mid_3rd#71835, Tack_Att_3rd#71836, Tkl%#71837, Tkl#71838, ... 131 more fields]
+- Union false, false
   :- Project [Player#71814, Squad#71815, Season#71816, Competition#71817, Born#71818, MP#71819, Starts#71820, Min#71821, Gls#71822, Ast#71823, YellowC#71824, RedC#71825, PrgC#71826, PrgP#71827, PrgR#71828, Fls#71829, Off#71830, Crosses#71831, Recov#72027, Aerialwon%#71833, Tack_Def_3rd#71834, Tack_Mid_3rd#71835, Tack_Att_3rd#71836

In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, trim, round, when
from pyspark.sql.types import FloatType
import builtins
position_metrics = {
    
    "Goalkeeper": {"Recov": 0.3, "Clearences": 0.2, "Pass_cmp%": 0.3, "GA": 0.3, "SoTA": 0.4, "Save%": 0.6, "CS%": 0.3, "Effective_Penalty_Saves": 0.2,
        "PSxG/SoT": 0.6, "PSxG": 0.6, "PSxG+/-": 0.5, "Launch%": 0.2, "Effective_Cross_stop": 0.5, "#OPA": 0.4
    },
    "Centre-Back": {
        "Int": 0.4, "Blocks": 0.3, "Clearences": 0.5, "Aerialwon%": 0.6,
        "Touch_Def_3rd": 0.2, "Tack_Def_3rd": 0.4, "PrgC": 0.1, "PrgP": 0.2, "Effective_Pass_Short": 0.2,
        "Effective_Pass_Medium": 0.2, "Effective_Pass_Long": 0.3,
        "Effective_Tkl": 0.5, "Block_Shots": 0.4, "Effective_Tackles_vsDrib": 0.3,
        "PrgDist": 0.1
    },
    "Left-Back":{ 
        "PrgC": 0.3, "PrgP": 0.2, "Crosses": 0.4, "Recov": 0.4, "Tack_Def_3rd": 0.4, "Effective_Tkl": 0.5, "Int": 0.5,
        "Block_Shots": 0.4, "Block_Pass": 0.4, "Clearences": 0.5, "Effective_Pass_Short": 0.2,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.1, "xA": 0.2,
        "A-xAG": 0.2, "CrsPA": 0.3, "Touch_Def_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.2,
        "PrgDist": 0.4, "Carries_Att_3rd": 0.2, "Touch_Live": 0.3, "Effective_Tackles_vsDrib": 0.5
    },
    "Right-Back": {
        "PrgC": 0.3, "PrgP": 0.2, "Crosses": 0.4, "Recov": 0.4, "Tack_Def_3rd": 0.4, "Effective_Tkl": 0.5, "Int": 0.5,
        "Block_Shots": 0.4, "Block_Pass": 0.4, "Clearences": 0.5, "Effective_Pass_Short": 0.2,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.1, "xA": 0.2,
        "A-xAG": 0.2, "CrsPA": 0.3, "Touch_Def_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.2,
        "PrgDist": 0.4, "Carries_Att_3rd": 0.2, "Touch_Live": 0.3, "Effective_Tackles_vsDrib": 0.5
    },
    "Defensive Midfield": {
        "Int": 0.4, "Blocks": 0.5, "Clearences": 0.3, "Aerialwon%": 0.3, "Touch_Def_3rd": 0.2,
        "Touch_Mid_3rd": 0.4, "Tack_Def_3rd": 0.4, "Effective_Pass_Short": 0.6,
        "Effective_Pass_Medium": 0.5, "Effective_Pass_Long": 0.5, "PrgC": 0.2, "PrgP": 0.3,
        "Effective_Tkl": 0.5, "Block_Shots": 0.3,"PrgDist": 0.2, "Recov": 0.4,
        "Tack_Mid_3rd": 0.3, "Touch_Live": 0.4
   } ,
    "Central Midfield": {
        "Ast": 0.2, "PrgC": 0.3, "PrgP": 0.4, "Recov": 0.3, "Aerialwon%": 0.2, "Tack_Def_3rd": 0.3, "Tack_Mid_3rd": 0.4,
        "Tack_Att_3rd": 0.4, "Effective_Tkl": 0.4, "Int": 0.4, "Block_Pass": 0.3,"Effective_Pass_Short": 0.6,
        "Effective_Pass_Medium": 0.6, "Effective_Pass_Long": 0.5, "xAG":0.2, "xA": 0.2, "A-xAG": 0.2, "Pass_cmp_Att_3rd": 0.3, "PPA": 0.4,
        "Touch_Def_3rd": 0.3, "Touch_Mid_3rd": 0.4, "Touch_Att_3rd": 0.3, "Effective_Tackles_vsDrib": 0.2,
        "PrgDist": 0.3, "Carries_Att_3rd": 0.4, "Touch_Live": 0.2, "Effective_Drib": 0.2
    },
    "Attacking Midfield": {
        "Ast": 0.3, "PrgC": 0.3, "PrgP": 0.4, "PrgR": 0.4, "Tack_Att_3rd": 0.3, "Tack_Mid_3rd": 0.2, "Block_Pass": 0.2,
        "Effective_Pass_Short": 0.5, "Effective_Pass_Medium": 0.5, "Effective_Pass_Long": 0.3, "xAG": 0.5, "xA": 0.4, "A-xAG":0.3, "Pass_cmp_Att_3rd": 0.5, "PPA": 0.4,
        "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.4, "Touch_Att_Pen": 0.2, "Effective_Drib": 0.3,
        "Effective_Tackles_vsDrib": 0.1, "PrgDist": 0.2, "Carries_Att_3rd": 0.3, "Carries_Att_Pen": 0.4,
        "Sh": 0.2, "SoT": 0.3, "G/Sh": 0.2, "Touch_Live": 0.2
    },
    "Right Midfield": {
        "Gls": 0.1, "Ast": 0.2, "PrgC": 0.4, "PrgP": 0.3, "PrgR": 0.3, "Crosses": 0.4, "Tack_Mid_3rd": 0.3, "Tack_Att_3rd": 0.3,
        "Effective_Tkl": 0.2, "Block_Pass": 0.2, "Effective_Pass_Short": 0.4,
        "Effective_Pass_Medium": 0.4, "Effective_Pass_Long": 0.2, "xAG": 0.3, "xA": 0.3, "A-xAG": 0.3, "Pass_cmp_Att_3rd": 0.4, "PPA": 0.5, "CrsPA": 0.5,
        "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.5, "Effective_Tackles_vsDrib": 0.2, "PrgDist": 0.4, "Carries_Att_3rd": 0.5, "Carries_Att_Pen": 0.4,
        "Sh": 0.2, "SoT": 0.2, "G/Sh": 0.2
   } ,
    "Left Midfield": {
        "Gls": 0.1, "Ast": 0.2, "PrgC": 0.4, "PrgP": 0.3, "PrgR": 0.3, "Crosses": 0.4, "Tack_Mid_3rd": 0.3, "Tack_Att_3rd": 0.3,
        "Effective_Tkl": 0.2, "Block_Pass": 0.2, "Effective_Pass_Short": 0.4,
        "Effective_Pass_Medium": 0.4, "Effective_Pass_Long": 0.2, "xAG": 0.3, "xA": 0.3, "A-xAG": 0.4, "Pass_cmp_Att_3rd": 0.4, "PPA": 0.5, "CrsPA": 0.5,
        "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.4, "Effective_Drib": 0.5, "Effective_Tackles_vsDrib": 0.2, "PrgDist": 0.4, "Carries_Att_3rd": 0.5, "Carries_Att_Pen": 0.4,
        "Sh": 0.2, "SoT": 0.2, "G/Sh": 0.2
    },
    "Left Winger": {
        "Gls": 0.2, "Ast": 0.3, "PrgC": 0.5, "PrgP": 0.4, "PrgR": 0.4, "Crosses": 0.3, "Tack_Att_3rd": 0.4, "Effective_Tkl": 0.2,
        "Block_Pass": 0.1, "Effective_Pass_Short": 0.3,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.2, "PPA": 0.5, "CrsPA": 0.6, "Touch_Att_3rd": 0.5,
        "Touch_Att_Pen": 0.5, "Effective_Drib": 0.6, "Effective_Tackles_vsDrib": 0.1,
        "PrgDist": 0.3, "Carries_Att_3rd": 0.4, "Carries_Att_Pen": 0.5, "xAG": 0.3, "xA": 0.3,
        "Sh": 0.4, "SoT": 0.3, "G/Sh": 0.3, "xG": 0.5, "npxG": 0.3, "G-xG": 0.3
    },
    "Right Winger": {
        "Gls": 0.2, "Ast": 0.3, "PrgC": 0.5, "PrgP": 0.4, "PrgR": 0.4, "Crosses": 0.3, "Tack_Att_3rd": 0.4, "Effective_Tkl": 0.2,
        "Block_Pass": 0.1, "Effective_Pass_Short": 0.3,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.2, "PPA": 0.5, "CrsPA": 0.6, "Touch_Att_3rd": 0.5,
        "Touch_Att_Pen": 0.5, "Effective_Drib": 0.6, "Effective_Tackles_vsDrib": 0.1,
        "PrgDist": 0.3, "Carries_Att_3rd": 0.4, "Carries_Att_Pen": 0.5, "xAG": 0.3, "xA": 0.3,
        "Sh": 0.4, "SoT": 0.3, "G/Sh": 0.3, "xG": 0.5, "npxG": 0.3, "G-xG": 0.3
    },
    "Second Striker": {
        "Gls": 0.3, "Ast": 0.3, "PrgC": 0.4, "PrgP": 0.5, "PrgR": 0.6, "Tack_Att_3rd": 0.4, "Block_Pass": 0.1,
        "Effective_Pass_Short": 0.4,
        "Effective_Pass_Medium": 0.3, "Effective_Pass_Long": 0.2, "xAG": 0.5, "xA": 0.4,
        "Pass_cmp_Att_3rd": 0.4, "PPA": 0.5, "Touch_Mid_3rd": 0.3, "Touch_Att_3rd": 0.6, "Touch_Att_Pen": 0.5,
        "Effective_Drib":0.4, "Effective_Tackles_vsDrib": 0.1, "PrgDist": 0.3,
        "Carries_Att_3rd": 0.3, "Carries_Att_Pen": 0.4,
        "Sh": 0.3, "SoT": 0.6, "G/Sh": 0.5, "xG": 0.6, "npxG": 0.4, "G-xG": 0.5, "Touch_Live": 0.2, "Effective_Tkl": 0.1
    },
    "Centre-Forward": {
        "Gls":0.3, "Ast":0.2, "PrgR": 0.3, "Aerialwon%": 0.5, "Tack_Att_3rd": 0.5, "Effective_Tkl": 0.4, "Block_Pass": 0.2,
        "Effective_Pass_Short": 0.3,
        "Effective_Pass_Medium": 0.2, "Effective_Pass_Long": 0.1, "xAG": 0.4,
        "xA": 0.4, "Pass_cmp_Att_3rd": 0.4, "Touch_Att_3rd": 0.5, "Touch_Mid_3rd": 0.2, "Touch_Att_Pen": 0.6,
        "Carries_Att_Pen": 0.5, "Effective_Drib": 0.4, 
        "Sh": 0.5, "SoT": 0.6, "G/Sh": 0.5, "xG": 0.7, "npxG": 0.5, "G-xG": 0.4
    }
}
def normalize_weights_by_position(position_metrics):
    normalized = {}
    for position, weights in position_metrics.items():
        total = sum(weights.values())
        if total == 0:
            raise ValueError(f"La posición '{position}' tiene una suma de pesos igual a cero.")
        normalized[position] = [
            (metric, builtins.round(weight / total, 4)) for metric, weight in weights.items()
        ]
    return normalized


In [85]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import DenseVector
from functools import reduce
from pyspark.ml.functions import vector_to_array


from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.sql.functions import col, lit
from functools import reduce

def calcular_performance_score_con_pesos(df, position_metrics_normalized):
    def procesar_posicion_x_temporada(df, pos, metric_weight_tuples):
        available_columns = df.columns
        seasons = [row["Season"] for row in df.select("Season").distinct().collect()]
        df_pos_season_list = []

        for season in seasons:
            df_season = df.filter((col("Pos") == pos) & (col("Season") == season))

            metrics = [m for m, _ in metric_weight_tuples if m in available_columns]
            weights = [w for m, w in metric_weight_tuples if m in available_columns]
            metric_weight_tuples_filtered = list(zip(metrics, weights))

            if not metric_weight_tuples_filtered:
                continue

            df_season = df_season.fillna(0, metrics)

            assembler = VectorAssembler(inputCols=metrics, outputCol="features_vec")
            df_season = assembler.transform(df_season)

            scaler = MinMaxScaler(inputCol="features_vec", outputCol="scaled_features")
            scaler_model = scaler.fit(df_season)
            df_season = scaler_model.transform(df_season)

            # ✅ convertir el vector a array usando librería oficial de PySpark ML
            df_season = df_season.withColumn("scaled_array", vector_to_array("scaled_features"))

            # calcular columnas ponderadas
            weighted_cols = []
            for i, (metric, weight) in enumerate(metric_weight_tuples_filtered):
                weighted_col_name = f"{metric}_weighted"
                df_season = df_season.withColumn(weighted_col_name, col("scaled_array")[i] * lit(weight))
                weighted_cols.append(col(weighted_col_name))

            if weighted_cols:
                df_season = df_season.withColumn("performance_score", sum(weighted_cols))
            else:
                df_season = df_season.withColumn("performance_score", lit(0.0))

            # eliminar columnas temporales
            cols_to_drop = ["features_vec", "scaled_features", "scaled_array"] + \
                           [f"{metric}_weighted" for metric, _ in metric_weight_tuples_filtered]
            df_season = df_season.drop(*cols_to_drop)

            df_season = df_season.withColumn("Evaluated_Position", lit(pos))
            df_pos_season_list.append(df_season)

        return reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), df_pos_season_list) if df_pos_season_list else df.limit(0)

    scored_dfs = []
    for pos, metric_weight_tuples in position_metrics_normalized.items():
        if metric_weight_tuples:
            scored_df = procesar_posicion_x_temporada(df, pos, metric_weight_tuples)
            if scored_df.count() > 0:
                scored_dfs.append(scored_df)

    return reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), scored_dfs) if scored_dfs else df.limit(0)


print("normalizar pesos...")
position_metrics_normalized = normalize_weights_by_position(position_metrics)
print("calculando el performance score...")
df = calcular_performance_score_con_pesos(df, position_metrics_normalized)
    

normalizar pesos...
calculando el performance score...


In [77]:
df_clean = spark.createDataFrame(df.rdd, schema=df.schema)
df_clean.show()


Py4JJavaError: An error occurred while calling o40286.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 940.0 failed 1 times, most recent failure: Lost task 0.0 in stage 940.0 (TID 837) (LAPTOP-SNOCNEUP.mshome.net executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:200)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:110)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:123)
	at org.apache.spark.sql.execution.python.BatchEvalPythonEvaluatorFactory.$anonfun$

In [75]:
print(df.columns)


['Player', 'Squad', 'Season', 'Competition', 'Born', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'YellowC', 'RedC', 'PrgC', 'PrgP', 'PrgR', 'Fls', 'Off', 'Crosses', 'Recov', 'Aerialwon%', 'Tack_Def_3rd', 'Tack_Mid_3rd', 'Tack_Att_3rd', 'Tkl%', 'Tkl', 'TklW', 'Int', 'Blocks', 'Block_Shots', 'Block_Pass', 'Clearences', 'Err', 'Pass_cmp', 'Pass_cmp%', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%', 'Pass_Long', 'Pass_cmp_Long%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA', 'Touch_Def_3rd', 'Touch_Mid_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen', 'Touch_Live', 'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen', 'fail_To_Gain_Control', 'Loss_Control_Tackle', 'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'Pos', 'Height', 'Value', 'GA', 'SoTA', 'Save%', 'CS%', 'PKatt', 'P_Save%', 'PSxG', 'PSxG/SoT', 'PSxG+/-', 'Launch%', 'Crosses_Opp', 'Crosses_stp%', '#OPA', 'Gls_per90', 'Ast_per90', 'PrgC_per90', 'PrgP_per90', 'Pr

In [74]:
df.show()

Py4JJavaError: An error occurred while calling o40125.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 938.0 failed 1 times, most recent failure: Lost task 0.0 in stage 938.0 (TID 835) (LAPTOP-SNOCNEUP.mshome.net executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:200)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:110)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:123)
	at org.apache.spark.sql.execution.python.BatchEvalPythonEvaluatorFactory.$anonfun$

In [90]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

def obtener_mejores_jugadores_por_posicion(df, temporada="2024-2025"):
    # Filtrar temporada y asegurarse de que performance_score no sea nulo
    df_filtrado = df.filter((col("Season") == temporada) & (col("performance_score").isNotNull()))

    # Crear ventana por Evaluated_Position y ordenar por performance_score descendente
    window_pos = Window.partitionBy("Evaluated_Position").orderBy(col("performance_score").desc())

    # Asignar ranking y quedarte con el top 1 por posición
    df_top = df_filtrado.withColumn("rank", row_number().over(window_pos)) \
                        .filter(col("rank") == 1) \
                        .drop("rank")

    return df_top

obtener_mejores_jugadores_por_posicion(df).select("Player", "Pos", "Squad", "Competition", "performance_score").show()


+----------------+------------------+---------------+--------------+------------------+
|          Player|               Pos|          Squad|   Competition| performance_score|
+----------------+------------------+---------------+--------------+------------------+
| bruno fernandes|Attacking Midfield| Manchester Utd|premier-league|0.7517643045792839|
|           pedri|  Central Midfield|      Barcelona|        laliga|0.7987493209855847|
|  patrick mainka|       Centre-Back|     Heidenheim|    bundesliga|0.6392226205033559|
|   kylian mbappe|    Centre-Forward|    Real Madrid|        laliga|0.7508262196707399|
|  moises caicedo|Defensive Midfield|        Chelsea|premier-league|0.6973172047647267|
|  sergio herrera|        Goalkeeper|        Osasuna|        laliga|0.6790347824362013|
|marcus tavernier|     Left Midfield|    Bournemouth|premier-league|0.8634158397671888|
|        raphinha|       Left Winger|      Barcelona|        laliga|0.7462308701971416|
|antonee robinson|         Left-